# Laboratorio No. 5 Optimización

## Librerias

In [2]:
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import time
import sympy as sym
from scipy.optimize import linprog, minimize

## Simplex

Función donde se implementa el algoritmo Simplex para optimización de problemas de programación lineal en la forma estandar.

* El código fue adapatado de: https://github.com/mmolteratx/Simplex



In [3]:
# ------------------------------------------------------------------
#                      Funciones Auxiliares
# ------------------------------------------------------------------

def printTableau(tableau, f):
  
    f = f[np.nonzero(f)]
    print("ind \t", end = "")
    for j in range(0, len(f)):
        print("x_" + str(j), end = "\t")
    for j in range(0, (len(tableau[0]) - len(f) - 2)):
        print("s_" + str(j), end = "\t")
    
    print()
    for j in range(0, len(tableau)):
        for i in range(0, len(tableau[0])):
            if(not np.isnan(tableau[j, i])):
                if(i == 0):
                    print(int(tableau[j, i]), end = "\t")
                else:
                    print(round(tableau[j, i], 2), end = "\t")
            else:
                print(end = "\t")
        print()

def getTableau(c, A, b):
  # Construct starting tableau
  c[0:len(c)] = -1 * c[0:len(c)]

  t2 = np.array([None, 0])
  numVar = len(c)
  numSlack = len(A)
    
  t2 = np.hstack(([None], c, [0]))
  
  basis = np.array([0] * numSlack)
  
  for i in range(0, len(basis)):
      basis[i] = numVar + i
      
  t1 = np.hstack((np.transpose([basis]), A, np.transpose([b])))
  
  tableau = np.vstack((t1, t2))
  
  tableau = np.array(tableau, dtype ='float')
  
  return tableau

# ------------------------------------------------------------------
#                      Funcion Simplex
# ------------------------------------------------------------------

def simplex(f, A, b, print_iter=True):
  for i in range(len(f)):
    f[i] = -1 * f[i]
    
  # Build Tableu
  tableau = getTableau(f, A, b)
      
  if(print_iter):
      print("Starting Tableau:")
      printTableau(tableau, f)
  
  # Assume initial basis is not optimal
  optimal = False

  # Keep track of iterations for display
  iter = 1

  while not optimal:
      
    if print_iter:
      print("--------------------------------------------------------------")
      print("ITERATION ", iter)
      printTableau(tableau, f)
        
    # Look for direction of decreased cost
    for cost in tableau[-1, 1:-1]:
      if cost < 0:
        optimal = False
        break
      optimal = True

    # If all directions result in decreased cost SBF is optimal
    if optimal: 
      break
      
    # nth variable enters basis, account for tableau indexing
    n = tableau[-1, 1:-1].tolist().index(np.amin(tableau[-1, 1:-1])) + 1 # PUEDE QUE NO SEA 2

    # Minimum ratio test, rth variable leaves basis 
    minimum = 99999
    r = -1
    
    for i in range(0, len(tableau)-1): 
      if (tableau[i, n] > 0):
        val = tableau[i, -1]/tableau[i, n]
        if val<minimum: 
          minimum = val 
          r = i
                    
    pivot = tableau[r, n] 
    
    print("Pivot Column:", n)
    print("Pivot Row:", r)
    print("Pivot Element: ", pivot)

    # Perform row operations 
    # Divide the pivot row with the pivot element 
    tableau[r, 1:] = tableau[r, 1:] / pivot 
    
    # Pivot other rows
    for i in range(0, len(tableau)): 
      if i != r:
        mult = tableau[i, n] / tableau[r, n]
        tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 

    # New basic variable 
    tableau[r, 0] = n - 1
    iter += 1
      
  if print_iter:
    print("--------------------------------------------------------------")
    print("Final Tableau reached in", iter, "iterations")
    printTableau(tableau, f)
  else:
      print("Solved")
      
  x = np.array([0] * len(f), dtype = float)
  # Save coefficients
  for key in range(0, (len(tableau))):
      if (tableau[key, 0] < len(f)):
          x[int(tableau[key, 0])] = tableau[key, -1]
  
  optimalValue = -1 * tableau[-1,-1]

  print("--------------------------------------------------------------")
  print("SOLUTION: ")
  print("Fobj Optimal Value: " + str(optimalValue))
  print("Solution: " + str(x))


##  Ejemplo

Ejemplo de como utilizar la función implementada en python.

Problema:
$$ \min f(x) = -3 x_1 -  x_2 - 3 x_3 $$

$$ \begin{array}{rl}
  2x_1 + x_2 + x_3 & \leq 2 \\
  x_1 + 2x_2 + 3x_3 & \leq 5 \\
  2x_1 + 2x_2 + x_3 & \leq 6 \\
  x_1,x_2, x_3 & \geq 0 \\
 \end{array}$$

 Forma Estándar:
 $$ \min f(x) = -3 x_1 -  x_2 - 3 x_3  $$

$$ \begin{array}{rl}
  2x_1 + x_2 + x_3 + y_1 & = 2 \\
  x_1 + 2x_2 + 3x_3 + y_2 & = 5 \\
  2x_1 + 2x_2 + x_3 + y_3 & = 6 \\
  x_1,x_2, x_3, y_1, y_2, y_3 & \geq 0 \\
 \end{array}$$

In [4]:
# Probar función
f = np.array([-3, -1, -3, 0, 0, 0])
A = np.array([[2, 1, 1, 1, 0, 0], [1, 2, 3, 0, 1, 0], [2, 2, 1, 0, 0, 1]])
b = np.array([2, 5, 6])

simplex(f,A,b)

Starting Tableau:
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
6	2.0	1.0	1.0	1.0	0.0	0.0	2.0	
7	1.0	2.0	3.0	0.0	1.0	0.0	5.0	
8	2.0	2.0	1.0	0.0	0.0	1.0	6.0	
	-3.0	-1.0	-3.0	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
6	2.0	1.0	1.0	1.0	0.0	0.0	2.0	
7	1.0	2.0	3.0	0.0	1.0	0.0	5.0	
8	2.0	2.0	1.0	0.0	0.0	1.0	6.0	
	-3.0	-1.0	-3.0	0.0	0.0	0.0	0.0	
Pivot Column: 1
Pivot Row: 0
Pivot Element:  2.0
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
0	1.0	0.5	0.5	0.5	0.0	0.0	1.0	
7	0.0	1.5	2.5	-0.5	1.0	0.0	4.0	
8	0.0	1.0	0.0	-1.0	0.0	1.0	4.0	
	0.0	0.5	-1.5	1.5	0.0	0.0	3.0	
Pivot Column: 3
Pivot Row: 1
Pivot Element:  2.5
--------------------------------------------------------------
ITERATION  3
ind 	x_0	x_1	x_2	s_0	s_1	s_2	
0	1.0	0.2	0.0	0.6	-0.2	0.0	0.2	
2	0.0	0.6	1.0	-0.2	0.4	0.0	1.6	
8	0.0	1.0	0.0	-1.0	0.0	1.0	4.0	
	0.0	1.4	0.0	1.2	0.6	0.0	5.4	
--------------------------------

# Explicación de algoritmo

> Planteamiento de problemas a y b en forma de PL, en primer lugar tenemos:  
Carmac Company es una fábrica de carros compactos y subcompactos. La producción de cada carro requiere una cierta cantidad de materia prima y mano de obra para su realización. Para los carros compactos la materia prima en (lbs) es 200 y 18 horas de mano de obra. Mientras que, para los carros subcompactos la materia prima en (lbs) es 150 y 20 horas de mano de obra. Se conoce que, el costo unitario de la materia prima es de $10 y el costo unitario de la mano de Obra es de $70. El total máximo disponible de libras es 80000 y el máximo de horas para mano de obra 9000.
- Problema:
$$ \max f(x_1, x_2) = 6740x_1 + 5100x_2$$

$$ \begin{array}{rl}
  200x_1 + 150x_2 \leq 80000 \\
  18x_1 + 20x_2 \leq 9000 \\
  x_1 \leq 1500 \\
  x_2 \leq 200 \\
  x_1,x_2\geq 0 \\
 \end{array}$$

- Forma Estándar:

$$ \min f(x_1, x_2) = -6740x_1 - 5100x_2$$

$$ \begin{array}{rl}
  200x_1 + 150x_2 + y_1 = 80000 \\
  18x_1 + 20x_2 + y_2 = 9000 \\
  x_1 + y_3 = 1500 \\
  x_2 + y_4 = 200 \\
  x_1,x_2,y_1,y_2,y_3,y_4\geq 0 \\
 \end{array}$$

In [5]:
# 

f1 = np.array([6740, 5100, 0, 0, 0, 0])
A1 = np.array([[200, 150, 1, 0, 0, 0],[18, 20, 0, 1, 0, 0],[1, 0, 0, 0, 1, 0],[0, 1, 0, 0, 0, 1]])
b1 = np.array([8000, 9000, 1500, 200])

simplex(-f1,A1,b1)

Starting Tableau:
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	200.0	150.0	1.0	0.0	0.0	0.0	8000.0	
7	18.0	20.0	0.0	1.0	0.0	0.0	9000.0	
8	1.0	0.0	0.0	0.0	1.0	0.0	1500.0	
9	0.0	1.0	0.0	0.0	0.0	1.0	200.0	
	-6740.0	-5100.0	0.0	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	200.0	150.0	1.0	0.0	0.0	0.0	8000.0	
7	18.0	20.0	0.0	1.0	0.0	0.0	9000.0	
8	1.0	0.0	0.0	0.0	1.0	0.0	1500.0	
9	0.0	1.0	0.0	0.0	0.0	1.0	200.0	
	-6740.0	-5100.0	0.0	0.0	0.0	0.0	0.0	
Pivot Column: 1
Pivot Row: 0
Pivot Element:  200.0
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
0	1.0	0.75	0.0	0.0	0.0	0.0	40.0	
7	0.0	6.5	-0.09	1.0	0.0	0.0	8280.0	
8	0.0	-0.75	-0.0	0.0	1.0	0.0	1460.0	
9	0.0	1.0	0.0	0.0	0.0	1.0	200.0	
	0.0	-45.0	33.7	0.0	0.0	0.0	269600.0	
Pivot Column: 2
Pivot Row: 0
Pivot Element:  0.75
--------------------------------------------------------------
ITERATION  3
ind 	x_0	x_1	s_0	s_1	s_2	s_3

> Planteamiento del problema enfocado en los helados consisten en el siguiente planteamiento:  
Cremheladito es una empresa que se caracteriza por producir helados y cuenta con cinco sabores disponibles: Chocolate, Vainilla, Chicle, Banano y Lulo. Por desgracia, la compañía tiene actualmente un déficit en el abastecimiento de leche, azúcar y crema por lo que la compañía decidió establecer una disponibilidad de 240 galones de leche, 190 libras de azúcar y 80 galones de crema disponibles al mes. La producción de un galón de helado de Chocolate consume 0.45 galones de leche, 0.5 libras de azúcar y 0.10 galones de crema. El galón de helado de Vainilla consume 0.5 galones de leche, 0.4 libras de azúcar y 0.15 galones de crema. El galón de helado de Chicle consume 0.4 galones de leche, 0.4 libras de azúcar y 0.3 galones de crema. El galón de helado de Banano consume 0.4 galones de leche, 0.4 libras de azúcar y 0.2 galones de crema. Y, por último, el galón de helado de Lulo consume 0.42 galones de leche, 0.2 libras de azúcar y 0.1 galones de crema. Las ganancias que se tienen al vender cada helado son $1.1, $1.0, $0.9, $0.92 y $1.2 respectivamente.

- Problema:

$$ \max f(h_i) = 1.1h_1 + h_2 + 0.9h_3 + 0.92h_4 +1.2h_5\;\;i \in [1,5]$$

$$ \begin{array}{rl}
  0.45h_1 + 0.5h_2 + 0.4h_3 + 0.4h_4 + 0.42h_5 \leq 240 \\
  0.5h_1 + 0.4h_2 + 0.4h_3 + 0.4h_4 + 0.2h_5  \leq 190\\
  0.1h_1 + 0.15h_2 + 0.3h_3 + 0.2h_4 + 0.1h_5  \leq 80 \\
  h_i\geq 0 \;\;i \in [1,5]\\
 \end{array}$$ 
 
- Forma Estándar:

$$ \min f(h_i) = -1.1h_1 - h_2 - 0.9h_3 - 0.92h_4 - 1.2h_5\;\;i \in [1,5]$$

$$ \begin{array}{rl}
  0.45h_1 + 0.5h_2 + 0.4h_3 + 0.4h_4 + 0.42h_5 + y_1 = 240 \\
  0.5h_1 + 0.4h_2 + 0.4h_3 + 0.4h_4 + 0.2h_5 + y_2  = 190\\
  0.1h_1 + 0.15h_2 + 0.3h_3 + 0.2h_4 + 0.1h_5 + y_3 = 80 \\
   y_1,y_2,y_3\geq 0 \\
  h_i\geq 0 \;\;i \in [1,5]\\
 \end{array}$$ 


In [6]:
# 

f2 = np.array([1.1, 1, 0.9, 0.92, 1.2, 0, 0, 0])
A2 = np.array([[0.45, 0.5, 0.4, 0.4, 0.42, 1, 0, 0],[0.5, 0.4, 0.4, 0.4, 0.2, 0, 1, 0],[0.1 , 0.15, 0.3, 0.2, 0.1, 0, 0, 1]])
b2 = np.array([240, 190, 80])

simplex(-f2,A2,b2)

Starting Tableau:
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	0.45	0.5	0.4	0.4	0.42	1.0	0.0	0.0	240.0	
9	0.5	0.4	0.4	0.4	0.2	0.0	1.0	0.0	190.0	
10	0.1	0.15	0.3	0.2	0.1	0.0	0.0	1.0	80.0	
	-1.1	-1.0	-0.9	-0.92	-1.2	-0.0	-0.0	-0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	0.45	0.5	0.4	0.4	0.42	1.0	0.0	0.0	240.0	
9	0.5	0.4	0.4	0.4	0.2	0.0	1.0	0.0	190.0	
10	0.1	0.15	0.3	0.2	0.1	0.0	0.0	1.0	80.0	
	-1.1	-1.0	-0.9	-0.92	-1.2	-0.0	-0.0	-0.0	0.0	
Pivot Column: 5
Pivot Row: 0
Pivot Element:  0.42
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
4	1.07	1.19	0.95	0.95	1.0	2.38	0.0	0.0	571.43	
9	0.29	0.16	0.21	0.21	0.0	-0.48	1.0	0.0	75.71	
10	-0.01	0.03	0.2	0.1	0.0	-0.24	0.0	1.0	22.86	
	0.19	0.43	0.24	0.22	0.0	2.86	0.0	0.0	685.71	
--------------------------------------------------------------
Final Tableau reached in 2 iterations
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1